In [1]:
# conda env : freecad_fem_env

import os
import sys

sys.path.append(os.path.join(os.environ["CONDA_PREFIX"], "lib"))

import FreeCAD
import Part
import ObjectsFem
import Mesh
import MeshPart
import Fem
import femmesh.femmesh2mesh
from femtools import ccxtools
import trimesh
import numpy as np
import datetime
from FreecadFemLib import *
import random


In [2]:
#init
doc = App.newDocument("doc")

trimesh_scene_meshes = []
constraint_scene_meshes = []
view_meshes = True

doc = document_setup(doc)
file_list = []
obj_file_list = []
save_path = "/scratch/aifa/DATA/FCAD_3D/new"
load_path = "/scratch/aifa/DATA/FCAD_3D/new"

In [3]:
# filename = "_20210517153229478077.obj"#init
# filename ="_202105_initBox.obj"#initBox
# file_list.append(filename)

In [4]:
file_list = os.listdir(load_path)
for f in file_list:
    if f.endswith('obj') and not f.startswith("."):
        obj_file_list.append(f)

In [5]:
### black list:
#'_20210517163929573370.obj'
# obj_file_list = ['_20210517163929573370.obj','_20210517174956901483.obj','_20210518001808400542.obj','resultmesh_fcad2.obj']

In [6]:
random.shuffle(obj_file_list)
len(obj_file_list)

151

In [ ]:

for fname in obj_file_list:
    fail = False
    
    doc,_, initMesh_Name, mesh_OK = init_shape(doc, load_path, fname)

    doc = remove_old_mesh_result_mesh(doc,name=initMesh_Name)    
    if not mesh_OK: continue

    force_position = np.random.randint(3,97)
    print(f"force position : {force_position}\n")

    doc , fem_ok = create_shape_femmesh(doc)    
    if not fem_ok: break
        
    doc , fixed_indx = set_constraint_fixed(doc)
    doc , force_indx = set_constraint_force_placement(doc, force_position=force_position)    
                                            
    if len(fixed_indx)==0 or len(force_indx)==0:
        print("force or fixed constraints are empty... no more analysis will be executed... ")
        break
        
    fixed_scene = add_constraints_to_scene_meshes(doc,fixed_indx,color='blue')
    force_scene = add_constraints_to_scene_meshes(doc,force_indx,color='red')
    constraint_scene_meshes.append(concat_meshes(fixed_scene,force_scene))

    doc = remove_old_shape_result_mesh(doc)
    
    row_data = []
    #     for force_val in range(15000,40000,10000):
    for force_val in range(2000,20000,1000):    
      
        doc = remove_old_mesh_result_mesh(doc)
        doc = set_constraint_force_value(doc,force_val)
        
        # run Analysis
        doc, fem_volume = run_analysis(doc)
        print(f".....................................{fem_volume}")
        if not fem_volume: fail = True  

        #------   create and export result mesh
        doc, out_mesh = create_mesh_from_result(doc)
        if (not out_mesh) or (not checkMesh(out_mesh)): fail = True   

        result_trimesh = Meshobj_to_trimesh(out_mesh)
        result_trimesh_decimated = result_trimesh.simplify_quadratic_decimation(2000)
        trimesh_decimated_topology = trimesh_to_mesh_topology(result_trimesh_decimated)
        result_trimesh_decimated = set_trimesh_color(result_trimesh_decimated)

        if view_meshes : trimesh_scene_meshes.append(result_trimesh_decimated)

        now = str(datetime.datetime.now())
        sav = "_"+now.replace("-","").replace(" ","").replace(":","").replace(".","")
        
        state_info = f"{initMesh_Name}_{force_position}_{force_val}"
        print(f"state_info : {state_info}")
        saved_filename = f"{sav}.obj"
        save_trimesh(result_trimesh_decimated,save_path,saved_filename)
        
        row_data.append([f"{initMesh_Name}.obj",force_position,force_val,saved_filename])
    
    if fail : break
    add_to_excel_file(row_data)
    
    #------   prepare shape for next state
    doc = remove_old_femmesh(doc)
    doc = clear_constraints(doc)
    doc = remove_old_solid(doc)

    doc = create_shape_from_mesh(doc,trimesh_decimated_topology) 
    
print("+++ done! +++")


 ******* _20210520151512219465.obj is loaded ******* 

     +++ Mesh is healthy!  +++ 
shape result mesh is removed...
mesh result mesh is removed...
force position : 55

femmesh created from shape
num of faces as fixed constraints: 31
num of faces as force constraints: 12
Analysis done successfully...
.....................................True
CCX_Results
Mesh by surface search method:  33.232374
mesh_out is converted to mesh...
     +++ Mesh is healthy!  +++ 
state_info : _20210520151512219465_55_2000
file is saved: /scratch/aifa/DATA/FCAD_3D/new/_20210521092753228984.obj
mesh result mesh is removed...
Analysis done successfully...
.....................................True
CCX_Results
Mesh by surface search method:  20.293508000000003
mesh_out is converted to mesh...
     +++ Mesh is healthy!  +++ 
state_info : _20210520151512219465_55_3000
file is saved: /scratch/aifa/DATA/FCAD_3D/new/_20210521092815986076.obj
mesh result mesh is removed...
Analysis done successfully...
............

Analysis done successfully...
.....................................True
CCX_Results
Mesh by surface search method:  28.917756999999938
mesh_out is converted to mesh...
     +++ Mesh is healthy!  +++ 
state_info : _20210520151448897131_66_6000
file is saved: /scratch/aifa/DATA/FCAD_3D/new/_20210521093702112536.obj
mesh result mesh is removed...
Analysis done successfully...
.....................................True
CCX_Results
Mesh by surface search method:  27.10842200000002
mesh_out is converted to mesh...
     +++ Mesh is healthy!  +++ 
state_info : _20210520151448897131_66_7000
file is saved: /scratch/aifa/DATA/FCAD_3D/new/_20210521093731943035.obj
mesh result mesh is removed...
Analysis done successfully...
.....................................True
CCX_Results
Mesh by surface search method:  26.341178000000014
mesh_out is converted to mesh...
     +++ Mesh is healthy!  +++ 
state_info : _20210520151448897131_66_8000
file is saved: /scratch/aifa/DATA/FCAD_3D/new/_2021052109380112893

In [ ]:
# print('initial mesh:')
# mesh_scene = generate_complete_meshes_scene(doc,trimesh_scene_meshes)
# mesh_scene.show()


In [ ]:
# print("the ith state - action (before running the analysis)")
# print(len(trimesh_scene_meshes), len(constraint_scene_meshes))

# i = len(trimesh_scene_meshes)
# state_scene = []

# state_scene.append(trimesh_scene_meshes[0])
# state_scene.append(constraint_scene_meshes[0])

# _scene = generate_complete_meshes_scene(doc,state_scene)
# _scene.show()

In [ ]:
# s = scene_view_normals(result_trimesh_decimated)
# s.show()

In [ ]:
### save_image() has problem with opengl (and jupyter)

# from PIL import Image
# import io 
# image = np.array(Image.open(io.BytesIO(trimesh.Scene(pnt).save_image(resolution=(200, 200)))))

# trimesh.Scene(pnt).show(line_settings={'point_size':130})